## Using hyperspy to convert an Oxford hyperspectal data cube from image to a vector format for DTSA-II

Exporting Oxford EDS maps to hyperspectral datacubes can only be described as kludgy and tedious. One first needs to convert the `AZtec` project to the older `INCA` project format and then export the the data as a `Ripple` file pair: a `.rpl` decriptor file and the binary output. The conversion dialog is shown below.

<figure>
  <img src="./png/Inca-conversion.png"
  alt="Drawing" style="width: 640px;"/>
<figcaption>Figure 1. - Inca conversion dialog</figcaption>
</figure>

This outputs an image cube that we need to convert. `Hyperspy` comes to the rescue. Here is how one does it.

First, set up an to work with an iPython 3.0 environment...

In [1]:
%matplotlib inline
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.hspy as hs
import jmToolsPy2 as jm2
# set the default figure size
plt.rcParams['figure.figsize'] = 11, 7

Hyperspy actualy has a function to fix during import the `.rpl` file generated by INCA which is written in an obsolete format. Now we can load the image cube and set the general title.

In [2]:
s = hs.load("Paint.rpl", stack = False, signal_type="EDS_SEM")

Loading as Image


Let's first look at the storage, thentake a look at the metadata as loaded

In [3]:
s

<Image, title: , dimensions: (2048|256, 192)>

In [4]:
s.metadata

├── General
│   ├── original_filename = Paint.rpl
│   └── title = 
└── Signal
    ├── binned = False
    ├── record_by = image
    ├── signal_origin = 
    └── signal_type = EDS_SEM

Note that there isn't much useful information here. Let's add some. We can only add to the directory tree branches that are there...

In [5]:
s.metadata.General.title = 'Paint Cross Section Map'

We really want to convert this to a vector cube. Hyperspy uses a Spectrum class. The conversion is painless.

In [6]:
s = s.as_spectrum(0)

Now let's take a peek at the storage and the metadata.

In [7]:
s

<EDSSEMSpectrum, title: Paint Cross Section Map, dimensions: (256, 192|2048)>

In [8]:
s.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 35.0
│       │       └── energy_resolution_MnKa = 130.0
│       └── tilt_stage = 0.0
├── General
│   ├── original_filename = Paint.rpl
│   └── title = Paint Cross Section Map
└── Signal
    ├── binned = True
    ├── record_by = spectrum
    ├── signal_origin = 
    └── signal_type = EDS_SEM

Note that the conversion is done but there is not much more metadata...

In [9]:
s.set_signal_type('EDS_SEM')

Let's take a look at what we have now. Note how the storage changed.

In [10]:
s

<EDSSEMSpectrum, title: Paint Cross Section Map, dimensions: (256, 192|2048)>

Let's look at the data type

In [11]:
s.data.dtype

dtype('<i2')

This tells us that it is a `little endian` integer with 2 bytes per pixel. We will want to change that if we want to import into DTSA-II. We would also like to calibrate the spectrum. The easiest way will be to use the sums spectrum from Oxford.

In [12]:
spec = hs.load('paint-sum-dtsa.msa')
spec.set_signal_type('EDS_SEM')
spec.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── EDS_det = u'SDDWLS'
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 35.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 14386.79
│       │       └── real_time = 26400.381
│       ├── beam_current = 0.0
│       ├── beam_energy = 20.0
│       └── tilt_stage = 0.0
├── General
│   ├── date = datetime.date(2011, 5, 19)
│   ├── original_filename = paint-sum-dtsa.msa
│   ├── time = datetime.time(5, 7)
│   └── title = u'paint-sum'
└── Signal
    ├── binned = True
    ├── record_by = spectrum
    ├── signal_origin = 
    └── signal_type = EDS_SEM

Note that we have a lot more information. Let's use it to calibrate the datacube.

In [13]:
s.get_calibration_from(spec)
s.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── EDS_det = u'SDDWLS'
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 35.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 14386.79
│       │       └── real_time = 26400.381
│       ├── beam_current = 0.0
│       ├── beam_energy = 20.0
│       └── tilt_stage = 0.0
├── General
│   ├── original_filename = Paint.rpl
│   └── title = Paint Cross Section Map
└── Signal
    ├── binned = True
    ├── record_by = spectrum
    ├── signal_origin = 
    └── signal_type = EDS_SEM

In [14]:
s.axes_manager

<Axes manager, axes: (<width axis, size: 256, index: 0>, <height axis, size: 192, index: 0>|<depth axis, size: 2048>)>

Note that this did not seem to pick up the calibration of the spectrum axis. We need to get these from the `.msa` file and from AZtec

In [15]:
s.axes_manager.navigation_axes[0].name = 'x'
s.axes_manager.navigation_axes[1].name = 'y'
units_name = '${\mu}m$'
s.axes_manager['x'].units = units_name
s.axes_manager['y'].units = units_name
s.axes_manager['x'].scale = 0.516
s.axes_manager['y'].scale = 0.516

The Ripple format is picky about data types and DTSA-II wants float. So let's change it and store the result.

In [16]:
s.change_dtype('float32')

In [17]:
s.save(filename='paint-vec.rpl')

The paint-vec.rpl file was created


Here is what we really need in the .rpl file

```
key value
width 256
height  192
depth 2048
offset  0
data-length 4
data-type float
byte-order  little-endian
record-by vector
ev-per-chan 10.0
detector-peak-width-ev  129.6
```

Note that `Hyperspy` puts the byte order as `dont-care`. DTSA-II did not like that. Try setting to `little-endian`.